# Example

There are some steps on how to use `energy_manager` in any project:

## 1. Installation

In [ ]:
#%pip install energy_manager


## 2. Imports

In [ ]:
import os
import seaborn as sns
import matplotlib.pyplot as plt
from energy_manager.classes.energy_manager import EnergyManager


## 3. Explore the class EnergyManager

In [ ]:
help(EnergyManager)


## 2. Define mandatory parameters

In [ ]:
user_city_name = "Nangis"
openweathermap_api_key = os.getenv("OPEN_WEATHER_API_KEY") # if you set your openweathermap api key as an environment variable
user_temperature = 19.5
user_dpe_usage = 1.5


## 3. Define optional parameters if you want

In [ ]:
user_insulation_factor = 1.5 # 1.0 is the default value set in the EnergyManager class


## 4. Instantiate the class EnergyManager with your set of parameters

In [ ]:
my_energy_manager = EnergyManager(
    city_name=user_city_name,
    openweathermap_api_key=openweathermap_api_key,
    temperature=user_temperature,
    dpe_usage=user_dpe_usage,
    insulation_factor=user_insulation_factor,
)

daily_expenses_df = my_energy_manager.get_daily_expenses()


## 5. Explore results of daily_expenses_df and do any manipulation you want

In [ ]:
daily_expenses_df.dtypes


In [ ]:
daily_expenses_df[["date_time", "weather_description", "building_type", "dpe_class", "option_0", "option_3"]].head()


## 6. Results meaning

Columns option_0, option_1, ..., option_4 values are energy costs in euros per square meter.

Settings of these 5 options provided by ENEDIS for EDF consumers are such as :

  - option_0 : energy price is the same at any hour of the day with a value of 25.16 euros/kwh;
  - option_1, ..., option_4 : energy price during peak hour is 27 euros/Kwh and is 20.68 euros/Kwh during for off-peak hours.


The first row of df_daily_expenses means that for the day 2024-12-26 from 00:00:00 to 01:00:00, if your house is an "Appartement" and has a DPE (diagnostic de performance energetique) of class A, your estimated energy cost for the option_0 is approximately less than 3 euros per square meter for the desired temperature of 19.5 degrees Celsius.

In [ ]:
mask = (daily_expenses_df["building_type"] == "Appartement") & (daily_expenses_df["dpe_class"] == "A")
new_df = daily_expenses_df[mask]


In [ ]:
new_df[["date_time", "weather_description", "dpe_class", "option_0", "option_3"]].head(n=10)


In [ ]:
plt.style.use('ggplot')
fig, ax = plt.subplots(figsize=(12, 7), dpi=100)
colors = sns.color_palette("husl", 5)

for i, option in enumerate(["option_0", "option_1", "option_2", "option_3", "option_4"]):
    ax.step(new_df.date_time.dt.hour, new_df[option], where="post", label=option,
            linewidth=2, color=colors[i])

ax.set_title("Energy Cost Over Time in Euros Per Square Meter", fontsize=16, fontweight='bold', pad=20)
ax.set_xlabel("Hour of the Day", fontsize=12, labelpad=10)
ax.set_ylabel("Energy Cost (€/m²)", fontsize=12, labelpad=10)
ax.set_xticks(range(0, 24, 2))
ax.set_xlim(0, 23)
ax.tick_params(axis='both', which='major', labelsize=10)
ax.legend(title="Options", title_fontsize='12', fontsize='10', loc='upper left', bbox_to_anchor=(1, 1))
ax.grid(True, linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()


##### _This could be an insight on how to choose your EDF energy contract based on your consumption hours._